In [1]:
import pandas as pd
import numpy as np 

import time
import sys
from datetime import datetime
from scipy import stats
from statsmodels.stats import stattools
import random 
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import multiprocessing as mp

pd.options.plotting.backend = "plotly"
np.seterr(divide = 'ignore', invalid = 'ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
from Labeling import*
from SampleWeight import *

In [3]:

import yfinance
import datetime as dt
def get_data():
  enddate = dt.datetime.strptime("2021-01-01", "%Y-%m-%d").date()
  startdate = enddate - dt.timedelta(days=360*2) # for 5 years

  data = yfinance.download(tickers="AAPL", period="700d", interval="1h")
  return data

df = get_data()


[*********************100%***********************]  1 of 1 completed


In [4]:
df["price"] = df["Close"]
df["size"] = df["Volume"]

In [5]:
df.index

DatetimeIndex(['2019-12-23 09:30:00-05:00', '2019-12-23 10:30:00-05:00',
               '2019-12-23 11:30:00-05:00', '2019-12-23 12:30:00-05:00',
               '2019-12-23 13:30:00-05:00', '2019-12-23 14:30:00-05:00',
               '2019-12-23 15:30:00-05:00', '2019-12-24 09:30:00-05:00',
               '2019-12-24 10:30:00-05:00', '2019-12-24 11:30:00-05:00',
               ...
               '2022-09-30 14:30:00-04:00', '2022-09-30 15:30:00-04:00',
               '2022-10-03 09:30:00-04:00', '2022-10-03 10:30:00-04:00',
               '2022-10-03 11:30:00-04:00', '2022-10-03 12:30:00-04:00',
               '2022-10-03 13:30:00-04:00', '2022-10-03 14:30:00-04:00',
               '2022-10-03 15:30:00-04:00', '2022-10-03 16:00:00-04:00'],
              dtype='datetime64[ns, America/New_York]', length=4889, freq=None)

In [6]:
ohlcv = timeBar(df, frequency = "1B")

ohlcv.dropna(inplace = True)
ohlcv.index = ohlcv.index.strftime("%Y-%m-%d")
ohlcv.index = [datetime.datetime.strptime(date, '%Y-%m-%d').date() for date in ohlcv.index]
ohlcv.index = pd.DatetimeIndex(ohlcv.index)
close = ohlcv.close
ohlcv.head()

,open,high,low,close,volume,ValueOfTrades,PriceMean,TickCount,PriceMeanLogReturn
2019-12-24,71.077499,71.086853,71.014572,71.086853,8893884,71.058683,71.052931,4,0.002091
2019-12-27,72.887497,72.887497,72.050003,72.457497,33163099,72.635032,72.553357,7,0.006404
2019-12-30,71.599998,73.067497,71.599998,72.911751,33805145,72.477243,72.642821,7,0.001232
2019-12-31,72.797501,73.364998,72.797501,73.364998,21387753,73.052841,73.081786,7,0.006025
2020-01-03,74.867500,74.987503,74.345001,74.345001,32570592,74.756242,74.720359,7,0.000573


In [7]:
rets, stds = dailyVol(close, span = 32)
rets.head()

2019-12-27    0.019281
2019-12-30    0.006269
2019-12-31    0.012525
2020-01-03    0.013358
2020-01-06    0.008070
Name: rets, dtype: float64

In [8]:
timeevents = eventscusum(close, 2)
timestamp = pd.Series(pd.NaT, index = timeevents)
side = pd.Series(1., index = timeevents)
target = stds.loc[timeevents] 
ptsl = [1, 1]
events1 = pd.concat({'timestamp': timestamp, 'target': target, 'side': side}, axis = 1).dropna(subset = ['target'])
events1.head(10)



,timestamp,target,side
2019-12-31,NaT,0.006404,1.0
2020-01-08,NaT,0.005180,1.0
2020-01-13,NaT,0.010757,1.0
2020-01-27,NaT,0.016820,1.0
2020-01-28,NaT,0.016203,1.0
2020-01-31,NaT,0.024360,1.0
2020-02-04,NaT,0.023596,1.0
2020-02-12,NaT,0.021066,1.0
2020-02-21,NaT,0.021493,1.0
2020-02-24,NaT,0.024474,1.0


In [9]:
out = tripleBarrier(close, events1, ptsl, molecule = timeevents)

In [10]:
t1 = verticalBarrier(close, timeevents, 21)
events2 = events(close, timeevents, ptsl = 1, target = stds, returnMin = 0.00, numThreads=1, timestamp = t1)
events2.head()

e:\Risklab\Sample Weight\Labeling.py:275: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df0=df0.append(i)


,timestamp,target
2019-12-31,2020-01-03,0.006404
2020-01-08,2020-01-09,0.005180
2020-01-13,2020-01-14,0.010757
2020-01-27,2020-01-28,0.016820
2020-01-28,2020-01-29,0.016203


In [11]:
out3 = label(events2, close)
out3.head()

,ret,bin
2019-12-31,0.013358,1.0
2020-01-08,0.021675,1.0
2020-01-13,-0.013503,-1.0
2020-01-27,0.028516,1.0
2020-01-28,0.020770,1.0


In [12]:
curr = concurrencyEvents(close.index,events2.timestamp,events2.index)

In [13]:
sample_weight = mpSampleWeight(events2.timestamp,curr,events2.index)

e:\Risklab\Sample Weight\SampleWeight.py:35: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  weight=pd.Series(index=molecule) # create pandas object fot weights


In [15]:
sample_weight

2019-12-31    1.000000
2020-01-08    1.000000
2020-01-13    1.000000
2020-01-27    0.750000
2020-01-28    0.750000
                ...   
2022-09-21    0.416667
2022-09-23    0.466667
2022-09-29    0.500000
2022-09-30    0.500000
2022-10-03    0.500000
Length: 314, dtype: float64

In [18]:
def RandomTimestamp(nObservation , nBars , maximumHolding):
    data = {'date': np.zeros(nObservation),
        'timestamp': np.zeros(nObservation)}
    timestamp = pd.DataFrame(data)
    for i in range(nObservation):
        date = random.randint(0,nBars)
        t = date + random.randint(1,maximumHolding+1)
        timestamp.date[i] = date
        timestamp.timestamp[i] = t
    timestamp = timestamp.sort_values(by=['date'])
    return timestamp
def MonteCarloSimulationforSequentionalBootstraps(nObservation , nBars , maximumHolding):
    timestamp = RandomTimestamp(nObservation , nBars , maximumHolding)
    
    barIndex  = np.array([i for i in range(int(np.max(timestamp.timestamp)) + 1)])
    indM=pd.DataFrame(0,index=barIndex,columns=range(timestamp.shape[0]))
    

    indexMatrix_ = IndexMatrix(barIndex,timestamp)
    
    rho = random.sample([i for i in range(np.shape(indexMatrix_)[1])] ,np.shape(indexMatrix_)[1])
    rho = np.array([int(col) for col in  rho])
    standardUniqueness = np.mean(averageUniqueness(indexMatrix_.iloc[:,rho]))
    rho = SequentialBootstrap(indexMatrix_,np.shape(indexMatrix_)[1])
    rho = np.array([int(col) for col in  rho])
    sequentioalUniqueness = np.mean(averageUniqueness(indexMatrix_.iloc[:,rho]))
    return standardUniqueness ,sequentioalUniqueness


In [19]:
def SimulateSequentionalVsStandardBootstrap(iteration,nObservation , nBars , maximumHolding):
    seqU = np.zeros(iteration)
    stdU = np.zeros(iteration)
    for i in range(iteration):
        print(i)
        standardUniqueness ,sequentioalUniqueness = MonteCarloSimulationforSequentionalBootstraps(nObservation , nBars , maximumHolding)
        seqU[i]=sequentioalUniqueness
        stdU[i] = standardUniqueness 
    return seqU,stdU
seqU , stdU = SimulateSequentionalVsStandardBootstrap(1000,10 , 100 , 5)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [20]:
import plotly.figure_factory as ff
import numpy as np

# Group data together
hist_data = [seqU , stdU ]

group_labels = ['seqU', 'stdU']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels)
fig.show()